# MNITSをもって、Flux.jlを入門する

[Julia Advent Calendar 2019](https://qiita.com/advent-calendar/2019/julialang)の13日目の記事です。

Juliaの界隈では一番有名と言っても過言ではないでしょう。

Flux.jlという機械学習ライブラリを紹介/入門しようと思います。

## 導入

早速導入していきましょう。
Pkg.jlを使って、インストールします。

In [118]:
import Pkg; Pkg.add("Flux");

 Resolving package versions...
  Updating `/opt/julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]


無事、インストールできました！！

## MNISTのデータセットを取り込む
以前、Juliaのslackのworkspaceにて、MNISTのデータセットはFluxで使えるか？という質問がありました。

意外と知られてないのかもしれないですね。

In [119]:
# 訓練データの読込
using Flux
using Flux.Data.MNIST
imgs = MNIST.images(:train);

例えば、訓練画像として、`train_X`には以下のものが入っています。

In [120]:
imgs[1]

これを訓練用のデータへ変換し、学習をしていきます。

# 学習モデルの定義

In [121]:
using Flux: onehotbatch
imgs = MNIST.images(:train)
train_X = hcat(float.(vec.(imgs))...)
labels = MNIST.labels(:train)
train_Y = onehotbatch(labels, 0:9)

# 学習モデルの定義
using Flux: Chain
using Flux: Dense
using NNlib: softmax
using NNlib: relu
layeri_1 = Dense(28^2, 100, relu)
layer1_2 = Dense(100, 100, relu)
layer2_o = Dense(100, 10)
model = Chain(layeri_1, layer1_2, layer2_o, softmax)

Chain(Dense(784, 100, relu), Dense(100, 100, relu), Dense(100, 10), softmax)

---

28x28の学習データから、10個の数字までの2層を定義します。

そして、訓練データをバッチサイズを32個に分割します。

ここは任意なので、まあ、いろいろ試してみてください。

In [122]:
# 訓練データを32個ずつに分割
using Base.Iterators: partition
batchsize = 32
serial_iterator = partition(1:size(train_Y)[2], batchsize)
train_dataset = map(batch -> (train_X[:, batch], train_Y[:, batch]), serial_iterator);

# 学習を実行！！
とりあえず、epochを10回程度回してみましょう。

In [123]:
# run training
using Flux: crossentropy
using Flux: @epochs
using Flux: ADAM
using Flux: train!
loss(x, y) = crossentropy(model(x), y)
opt = ADAM()
epochs = 10
@epochs epochs train!(loss, Flux.params(model), train_dataset, opt)

┌ Info: Epoch 1
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 2
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 3
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 4
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 5
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 6
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 7
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 8
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 9
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99
┌ Info: Epoch 10
└ @ Main /opt/julia/packages/Flux/oX9Pi/src/optimise/train.jl:99


---

GPU使ってないので、こんなんでも割としんどいですね。

Colabとかがいいのかもしれません。無料でGPU枠があるので。

訓練が終わったら、モデルを保存する、ということになっていきます。

## 学習済みモデルの保存と読み込み
この程度なら必要ないですけど、普通はしますよね

https://github.com/FluxML/Flux.jl/blob/master/docs/src/saving.md　でも書いてあるように、BSON.jlというパッケージを使います。

In [91]:
# モデルの保存
using BSON: @save
pretrained = cpu(model)
weights = params(pretrained)
@save "pretrained.bson" pretrained
@save "weights.bson" weights

UndefRefError: UndefRefError: access to undefined reference

In [146]:
# モデルのロード
using BSON: @load
@load "pretrained.bson" pretrained
@load "weights.bson" weights

EOFError: EOFError: read end of file

---

ふむ。

どこかの重みがNaNになってるみたいですね。

NaNの部分を0へ変えたらうまくいくのでしょうか。。。？

うーん。。。
割とhttps://github.com/FluxML/Flux.jl/blob/master/docs/src/training/training.md　の通りにしてるんですけどねえ。

とりあえず、このまま行きますか。

# 検証データの取り込み

In [124]:
# テストデータの読込
imgs = MNIST.images(:test)
test_X = hcat(float.(vec.(imgs))...)
labels = MNIST.labels(:test);

基本的に、trainデータの取り込みと同じです。

In [125]:
imgs[1]

こんな感じですね。
これが7と判定されるとありがたいですねー

## 訓練済みモデルを用いた推論の実行

訓練済みモデル`pretrained`を用いて、推論を実行します。

さて、試しに、上の1番目の画像の推論することにします。

In [132]:
using Flux: onecold
onecold(pretrained(test_X[:,1]))

1

In [133]:
pretrained(test_X[:,1])

10-element Array{Float32,1}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

---

なん。。。だと。。。

わけわかんねえ。

くそう。まだまだ、勉強が足りないですね。
一旦、制度だけでも求めてみますか。

In [134]:
using Statistics: mean
mean(onecold(pretrained(test_X[:,1])) .== labels)

0.1135

---
うん。どうも、これ。たまたま正解ラベルに１がついたものだけが成功した結果みたいですね。

まじなんなんだ。

# 重みを用いた推論
あー、上の失敗したから、モチベ上がんねえ。

おまけに、NaNがあるのに、できるのかしら。

やるだけやってみます💪

In [144]:
p_layeri_1 = Dense(28^2, 100, relu)
p_layer1_2 = Dense(100, 100, relu)
p_layer2_o = Dense(100, 10)
p_model = Chain(layeri_1, layer1_2, layer2_o, softmax)
Flux.loadparams!(p_model, weights)
mean(onecold(model(test_X)) .== labels)

0.1135

---

はい。ダメでした。あーくそ。

全然ダメですね。まじでわかんねえ。

ソースコード追うしかないな。

## 終わりに

全く、成功しなかったので、引き続き勉強していきたいとおもます！

もし、ここはこうしたほうがいいで！とか

ここ間違ってるわ、ぼけえ

とか教えてくれる心優しい方々、[issue](https://github.com/TsuMakoto/study_fluxjl/issues)で投げてください！